<a href="https://colab.research.google.com/github/yogyata-04/Handwritten-Emoji-Letter-Captcha-Detection/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [285]:
import cv2
from google.colab.patches import cv2_imshow
import tensorflow as tf
import numpy as np

In [286]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [287]:
model_path1 = '/content/drive/MyDrive/mosaic/model_emoji_dataset_final.h5'
model_emoji= tf.keras.models.load_model(model_path1)
model_path2='/content/drive/MyDrive/mosaic/model_letter_dataset_final.h5'
model_letter=tf.keras.models.load_model(model_path2)

In [ ]:
print("Enter the path of the image")
a = input()

In [ ]:
#read and thresholded image
image=cv2.imread(a)
image = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
_,thresh= cv2.threshold(image,100, 255, cv2.THRESH_BINARY)

img=cv2.copyMakeBorder(thresh,200,200,0,0,cv2.BORDER_CONSTANT,value=[255,255,255])
img.shape

#erode image
kernal=cv2.getStructuringElement(cv2.MORPH_RECT,(3,13))
erode=cv2.erode(img,kernal,iterations=10)

#collecting the required contours in cnt_new
cnts,_=cv2.findContours(erode, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
cnt_new=[]
for num in range(len(cnts)):
  if(cv2.contourArea(cnts[num])>2000):
    cnt_new.append(cnts[num])

#appending all the contour values in x and y matrix which updates from beginning after every iteration
#then we have appended values in matrices xi,yi,xf,yf where xi is minimum x value of contour points and xf is maximum x value of contour points
#and same for yi and yf as well
xi=[]
xf=[]
yi=[]
yf=[]
for num in range(len(cnt_new)):
  x=[]
  for i in range(len(cnt_new[num])):
    b=cnt_new[num][i][0][0]
    x.append(b)
  #print(x)
  y=[]
  for i in range(len(cnt_new[num])):
    b=cnt_new[num][i][0][1]
    y.append(b)

  #print(cnt_new[num].shape)

  y=np.array(y)
  x=np.array(x)
  y_=min(y)
  y_h=max(y)
  x_=min(x)
  x_h=max(x)
  xi.append(x_)
  yi.append(y_)
  xf.append(x_h)
  yf.append(y_h)

#swapping values of coordinates on the basis of xi to get required order of images
for i in range(len(cnt_new)):
  for j in range(i):
    if(xi[i]<xi[j]):
      temp=xi[i]
      xi[i]=xi[j]
      xi[j]=temp
      temp=xf[i]
      xf[i]=xf[j]
      xf[j]=temp
      temp=yi[i]
      yi[i]=yi[j]
      yi[j]=temp
      temp=yf[i]
      yf[i]=yf[j]
      yf[j]=temp

#appending images in parts on the basis of the swapped order above
parts=[]
for i in range(len(cnt_new)-1):
  cropped_image = image[yi[i+1]-160:yf[i+1]-240,xi[i+1]:xf[i+1]]
  parts.append(cropped_image)

parts_new=[]
img_data=[]
for i in range(len(cnt_new)-1):

  #_,thresh= cv2.threshold(parts[i],150, 255, cv2.THRESH_BINARY)
  kernal=cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
  erode=cv2.erode(parts[i],kernal,iterations=1)
  image=cv2.resize(erode, (28, 28),interpolation = cv2.INTER_AREA)
  image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
  image = cv2.flip(image,0)
  parts_new.append(image)
  image=np.array(image)
  image = image.astype('float32')
  img_data.append(image)
  cv2_imshow(image)
img_data = np.array(img_data)
img_data[img_data>100] = 255
img_data[img_data<=100] = 0
img_data=(255-img_data)/255

In [290]:
my_dict_letter= {0:'A',1:'B',2:'D' ,3:'E',4:'F',5:'H',6:'J',7:'K',8:'L',9:'M',10:'N',11:'P',12:'R',13:'S',14:'T',15:'W',16:'X',17:'Y',18:'Z'}

In [ ]:
#prediction of captcha
prediction=[]
for i in range(img_data.shape[0]):
  image=img_data[i]
  reshaped_arr=image.reshape((1, 28, 28, 1))
  test1= model_emoji.predict(reshaped_arr)
  test2= model_letter.predict(reshaped_arr)
  result_confidence1=np.max(test1)
  result_confidence2=np.max(test2)
  result1= np.argmax(test1,axis = 1)
  result2= np.argmax(test2,axis = 1)
  print(result1)
  print(result2)
  print(result_confidence1)
  print(result_confidence2)
  if(result_confidence2>0.95):
    #we consider it is emoji
    prediction.append(my_dict_letter[result2[0]])
  else:
    image = cv2.flip(image,0)
    image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    prediction.append(result1[0])

print(prediction)